In [1]:
from langchain.chat_models import AzureChatOpenAI, ChatVertexAI
from langchain.embeddings import OpenAIEmbeddings, VertexAIEmbeddings
from langchain.schema import (
    AIMessage, 
    HumanMessage, 
    SystemMessage
)
from langchain.text_splitter import RecursiveCharacterTextSplitter,CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.docstore.document import Document
from langchain.document_loaders import DirectoryLoader
from langchain.chains import AnalyzeDocumentChain
from langchain.chains.summarize import load_summarize_chain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
import os

In [2]:
# IF you are running this please just insert the os keys that was given

import sys
sys.path.append('/home/dark/VS-CodePythonProjects/Gamefication-Retirement/KeysAI')

import OpenAIKeysTIAA

In [3]:
with open('../Transaction_File_Creator/financial_game_summary.txt', 'r') as f:
    data_to_summarize = f.read()

In [4]:
import openai
openai.api_type='azure'
openai.api_base = os.environ["OPENAI_API_HOST"]
openai.api_key = os.environ["OPENAI_API_KEY"]

base_model_name="tiaa-gpt-4-32k"
chosen_llm = AzureChatOpenAI(
        openai_api_base=os.environ["OPENAI_API_HOST"],
        openai_api_version=os.environ["OPENAI_API_VERSION"],
        deployment_name=base_model_name,
        openai_api_key=os.environ["OPENAI_API_KEY"],
        openai_api_type="azure",
        temperature=0
    )
embedding_model_name = 'tiaa-text-embedding-ada-002'
chosen_embedding_model = OpenAIEmbeddings(
        deployment=embedding_model_name,
        openai_api_base=os.environ["OPENAI_API_EMBEDDING_HOST"],
        openai_api_version= os.environ["OPENAI_API_VERSION"],
        openai_api_key=os.environ["OPENAI_API_EMBEDDING_KEY"],
        openai_api_type="azure",
        chunk_size=16
    )
finalAnswerTheQuestionModel = AzureChatOpenAI(
        openai_api_base=os.environ["OPENAI_API_HOST"],
        openai_api_version=os.environ["OPENAI_API_VERSION"],
        deployment_name=base_model_name,
        openai_api_key=os.environ["OPENAI_API_KEY"],
        openai_api_type="azure",
        temperature=0,
        verbose = True
    )
condensingTheQuestionModel = AzureChatOpenAI(
        openai_api_base=os.environ["OPENAI_API_HOST"],
        openai_api_version=os.environ["OPENAI_API_VERSION"],
        deployment_name=base_model_name,
        openai_api_key=os.environ["OPENAI_API_KEY"],
        openai_api_type="azure",
        temperature=0, 
        verbose = True
    )
memory = ConversationBufferMemory(
    output_key='answer',
    memory_key='chat_history',
    return_messages=True
    )


In [5]:
def vectorstore(text):
    new_doc = Document(page_content=text, metadata=[])
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    chunked_docs = text_splitter.split_documents([new_doc])
    vectorstore = FAISS.from_documents(chunked_docs, chosen_embedding_model)
    return(vectorstore)

In [6]:
current_vector=vectorstore(data_to_summarize)

In [7]:
qa = ConversationalRetrievalChain.from_llm(
    llm = finalAnswerTheQuestionModel,
    retriever = current_vector.as_retriever(),
    condense_question_llm = condensingTheQuestionModel,
)
source_with_qa = ConversationalRetrievalChain.from_llm(
    llm = finalAnswerTheQuestionModel,
    retriever = current_vector.as_retriever(
                                         search_kwargs={
                                            "k": 8
                                            }),
    condense_question_llm = condensingTheQuestionModel,
    memory=memory,
    return_source_documents=True,
    verbose=False
)

In [8]:
chat_history = []
query = f"Give A break down for all users and there financial scores, give an in depth overview on why their scores are the way they are."
result = source_with_qa({"question": query, "chat_history": chat_history})
ans = result['answer']

print(ans)

1. Account ID: pznbjQnW6KienZgWVB7EiMQJezZLo1hpqangP
   - Final Score for the month: 68
   - This user lost points for exceeding the essential budget. The score from savings was 10, but there is no information about the score from essentials and non-essentials budget compliance.

2. Account ID: MAD9LQDKnrC6ybzMWgE9fl6XoRx8eBtL5qyxw
   - Final Score for the month: 68
   - This user also lost points for exceeding the essential budget. The score from savings was 10, but there is no information about the score from essentials and non-essentials budget compliance.

3. Account ID: 1GErvgE9ANFdK46JejG7iJDVnGjg9qCpaBo7n
   - Final Score for the month: 68
   - This user had several transactions, both essential and non-essential. They lost points for exceeding the essential budget in multiple months. The base score was 60, with a score of -4 from essentials budget compliance, a score of 2 from non-essentials budget compliance, and a score of 10 from savings. Despite exceeding the essential budge

In [9]:
chat_history = []
query = f""
result = source_with_qa({"question": query, "chat_history": chat_history})
ans = result['answer']

print(ans)

I'm sorry, but I can only provide information on two users based on the context provided.

1. Account ID: pznbjQnW6KienZgWVB7EiMQJezZLo1hpqangP
   - Base Score: 60
   - Score from Essentials Budget Compliance: -4 (This user lost points for exceeding the essential budget)
   - Score from Non-Essentials Budget Compliance: 2 (This user complied with the non-essential budget)
   - Score from Savings: 10
   - Final Score for the month: 68

2. Account ID: MAD9LQDKnrC6ybzMWgE9fl6XoRx8eBtL5qyxw
   - The specific breakdown for this user is not provided in the context, but it is mentioned that they lost points for exceeding the essential budget. Their final score for the month is also 68.

Without more specific information, I can't provide a more in-depth overview.
